In [1]:
import pandas as pd
# 9人篇
data = pd.read_csv('gaze_1017.csv')


In [2]:
data.head()

,gaze_x,gaze_y,gaze_t
0,"499.9281916929646,500.7827776075044,505.779561...","500.01446883798474,499.7674575377377,499.27408...","15.5,1094.5,1138.7999999523163,1167.1999999284..."
1,"500.0809183161369,501.2473190313305,505.520843...","499.94212464806105,500.00284609397926,501.3383...","18.899999999441206,7789.5,7835.700000000186,78..."
2,"976.0934688438953,940.3889573899594,902.095533...","427.87624362040066,394.97962288910423,441.4959...","16466.39999999944,16784.299999999814,16853.299..."
3,"500.11682246965455,500.8400332489199,502.42095...","500.08466949635516,499.9072253333237,499.27497...","23.299999999813735,1134.0999999996275,1188.399..."
4,"499.94855524272094,503.02118947543767,506.5767...","500.1119995236597,500.8045022214369,500.412437...","15.699999999254942,1119.1000000014901,1164.900..."


In [11]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.AvgPool1d(7,stride=1,padding=3,count_include_pad=False)
        )
    def forward(self,input):
        return self.model(input)

In [4]:
# 读入数据
gaze_x = data['gaze_x']
gaze_x = gaze_x[0].split(',')
gaze_x = list(map(float,gaze_x))
gaze_x = list(map(int,gaze_x))

In [29]:
import numpy as np
data = gaze_x
data = np.array(data).reshape(1, -1)
data.shape

(1, 1021)

In [4]:
# 过滤器（ground truth)
from scipy import signal

def preprocess_data(data, win):
    data = meanFilter(data, win)
    # data = meanFilter(data, win)
    return data

def meanFilter(data, win):
    length = len(data)
    res = np.zeros(length)
    for i in range(length):
        s, n = 0, 0
        for j in range(i - win // 2, i + win - win // 2):
            if j < 0 or j >= length:
                continue
            else:
                s += data[j]
                n += 1
        res[i] = s / n
    return res

In [69]:
filter_result = preprocess_data(gaze_x,7)

In [70]:
cnn = CNN()
data = torch.tensor(data,dtype=torch.float32)
cnn_result = cnn.forward(data)
print(cnn_result.size())
cnn_result = torch.squeeze(cnn_result,0)
cnn_result = cnn_result.numpy()
print(cnn_result)

torch.Size([1, 1021])
[ 505.       508.2      508.66666 ... 1327.3334  1357.      1382.25   ]


D:\qxy\software-ide\ana\envs\torch-cpu\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [71]:
cnt = 0
print(len(filter_result))
print(len(cnn_result))
for i in range(len(filter_result)):
    if int(filter_result[i]) != int(cnn_result[i]):
        cnt += 1
cnt

1021
1021


0

In [2]:
import torch
import torch.nn as nn
class Filter(nn.Module):
    def __init__(self):
        super(Filter, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(in_channels=1,out_channels=1,kernel_size=7,stride=1,padding=3)
        )
    def forward(self,input):
        return self.model(input)



In [ ]:
from torch.utils.data import Dataset,DataLoader
class MyDataset(Dataset):
    def __init__(self,X,Y):
        self.data_x = torch.FloatTensor(X,)
        self.data_y = torch.FloatTensor(Y,)
        self.len = X.shape[0]
        pass
    def __getitem__(self, item):
        return self.data_x[item],self.data_y[item]
    def __len__(self):
        return self.len
batch_size = 16

In [6]:
import pandas as pd
# 9人篇
def load_csv(filename):
    data = pd.read_csv(filename)
    gaze_x = data['gaze_x']
    gaze_x = gaze_x[0].split(',')
    gaze_x = list(map(float,gaze_x))
    gaze_x = list(map(int,gaze_x))
    gazes = []
    labels = []
    for x in gaze_x:
        if len(x) > 300:
            x = x[0:300]
            gazes.append(x)
            labels.append(preprocess_data(x,7))
    assert len(gazes) == len(labels)
    return gazes,labels
load_csv('gaze_1017.csv')


TypeError: object of type 'int' has no len()

In [20]:
from torch.utils.data import DataLoader
import os
# 读取数据
datas = DataLoader(torch_data, batch_size=6, shuffle=True, drop_last=False, num_workers=0)
datas

In [21]:
for i, data in enumerate(datas):
	# i表示第几个batch， data表示该batch对应的数据，包含data和对应的labels
    print("第 {} 个Batch \n{}".format(i, data))

RuntimeError: DataLoader worker (pid(s) 9756, 6224) exited unexpectedly

In [87]:

import pandas as pd

data = pd.read_csv('gaze_1017.csv')
gaze_data = []
labels = []
cnt = 0
for i,item in enumerate(data['gaze_x']):
    if not pd.isnull(item):
        gaze_x = item.split(',')
        gaze_x = list(map(float,gaze_x))
        if len(gaze_x)>300:
            gaze_x = gaze_x[0:300]
            gaze_data.append(gaze_x)
            labels.append(preprocess_data(gaze_x,7))
print(gaze_data)
print(labels)

[[499.9281916929646, 500.7827776075044, 505.7795611310622, 516.9790333229427, 521.2837594472613, 511.91814026049894, 512.5822363823858, 489.95386482593346, 464.21056400233243, 438.77685158155157, 432.26719199267495, 482.87814483256847, 481.3093509163213, 478.72435378697406, 484.8322230216256, 451.2167430119804, 435.680617383971, 452.8986154744032, 439.9040722042666, 418.2626906282669, 385.49843821795736, 397.5603658991169, 396.33773475695494, 410.47588334862314, 413.08047228765287, 404.18518091731136, 404.471674352555, 430.9928774855537, 431.8991162778794, 434.5861026303498, 439.5839577130041, 467.200781501859, 476.565457083092, 474.16963578460604, 502.28610288965837, 495.51098536711413, 470.7260717733003, 494.75199278197096, 522.8480768330498, 522.8798594578501, 510.9645994989255, 502.12280657600394, 488.88934043166864, 499.36470636281547, 497.01880787585367, 511.0771373336425, 519.2552709717523, 539.7488354413134, 501.2654871852958, 526.2148663836008, 498.79832081734014, 493.96160409

In [88]:

import torch
import numpy as np


# 定义GetLoader类，继承Dataset方法，并重写__getitem__()和__len__()方法
class GetLoader(torch.utils.data.Dataset):
	# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return len(self.data)

# 随机生成数据，大小为10 * 20列
source_data = np.array(gaze_data)
# 随机生成标签，大小为10 * 1列
source_label = np.array(labels)
# 通过GetLoader将数据进行加载，返回Dataset对象，包含data和labels
torch_data = GetLoader(source_data, source_label)
source_data,source_label

(array([[ 499.92819169,  500.78277761,  505.77956113, ...,  976.43641633,
          992.64416779, 1001.04660654],
        [ 499.94855524,  503.02118948,  506.57674605, ...,  430.37371059,
          415.31622635,  408.87884632],
        [ 500.2411473 ,  500.96050647,  498.25277325, ...,  429.99266631,
          485.77039668,  482.70008188],
        ...,
        [1045.24286715, 1025.00045494, 1004.63299456, ..., 1238.64165171,
         1236.11692322, 1234.47044212],
        [ 499.6211308 ,  499.64225569,  500.57263331, ..., 1547.4126541 ,
         1559.22973623, 1526.75810899],
        [1125.84953985, 1116.32664921, 1046.14577371, ...,  529.06583422,
          723.42005774, 1060.91999415]]),
 array([[ 505.86739094,  508.95066464,  509.44524391, ...,  978.38721906,
          985.66879476,  990.92051894],
        [ 506.32805364,  509.45549152,  510.70472774, ...,  526.7538549 ,
          488.64150884,  454.49590893],
        [ 498.91714087,  497.00911498,  496.60669601, ...,  412.41860618,

In [89]:
from torch.utils.data import DataLoader
import os
# 读取数据
datas = DataLoader(torch_data, batch_size=6, shuffle=True, drop_last=False, num_workers=0)
datas

In [90]:
for i, data in enumerate(datas):
	# i表示第几个batch， data表示该batch对应的数据，包含data和对应的labels
    print("第 {} 个Batch \n{}".format(i, data))

第 0 个Batch 
[tensor([[ 499.9084,  498.7208,  493.1165,  ..., 1216.1109, 1235.4893,
         1253.4159],
        [ 957.9296,  870.7541,  821.6854,  ...,  739.1493,  761.7117,
          808.3354],
        [ 904.9043,  872.6360,  783.2129,  ...,  874.1408,  880.9165,
          871.2505],
        [1098.1707,  949.9972,  815.7106,  ..., 1039.5199, 1130.0296,
         1253.6942],
        [ 500.3960,  503.3709,  505.0346,  ...,  357.4613,  378.3938,
          408.3999],
        [1125.8495, 1116.3266, 1046.1458,  ...,  529.0658,  723.4201,
         1060.9200]], dtype=torch.float64), tensor([[ 493.2824,  488.7088,  483.1188,  ..., 1213.7262, 1220.4679,
         1226.0760],
        [ 860.1049,  853.3829,  850.2254,  ...,  725.0606,  735.7375,
          756.1385],
        [ 828.1323,  819.4164,  805.6491,  ...,  875.8052,  876.0329,
          874.6291],
        [ 895.6800,  832.3538,  770.6546,  ..., 1059.8952, 1079.1224,
         1104.6555],
        [ 502.6804,  497.9865,  495.6249,  ...,  383.1

In [146]:
import torch
import torch.nn as nn
class Filter_cnn(nn.Module):
    def __init__(self):
        super(Filter_cnn, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(in_channels=1,out_channels=1,kernel_size=7,stride=1,padding=3)
        )
    def forward(self,input):
        return self.model(input)

In [147]:
net = Filter_cnn()
print(net)

Filter_cnn(
  (model): Sequential(
    (0): Conv1d(1, 1, kernel_size=(7,), stride=(1,), padding=(3,))
  )
)


In [148]:
import torch.optim as optimizer
# 定义损失函数和优化器
criterion_loss = nn.CrossEntropyLoss()
opt = optimizer.Adam(net.parameters(), lr=0.001)

# 训练模型
Loss_list = []
correct_list = []
for epoch in range(100):
    correct = 0
    running_loss = 0
    for i,data in enumerate(datas):
        train_x,train_y = data
        train_x = train_x.to(torch.float32)
        train_y = train_y.to(torch.float32)
        train_x = train_x.unsqueeze(dim=1)
        outputs = net(train_x)
        outputs = outputs.squeeze(dim=1)
        loss = criterion_loss(outputs,train_y)
        print(loss)
        opt.zero_grad()
        loss.backward()
        opt.step()

        predicted = torch.max(outputs.data, 1)[1]
        y = torch.max(train_y.data, 1)[1]
        running_loss += loss.item()
        correct += (predicted == y).sum()
    Loss_list.append(running_loss / 11)
    correct_list.append(correct / 11)


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.DoubleTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [103]:
for name, param in net.named_parameters():
    print(name,'-->',param.type(),'-->',param.dtype,'-->',param.shape)



model.0.weight --> torch.FloatTensor --> torch.float32 --> torch.Size([1, 1, 7])
model.0.bias --> torch.FloatTensor --> torch.float32 --> torch.Size([1])


In [150]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./logs')
for i in range(100):
    loss = i
    writer.add_scalar("loss",loss,i)
writer.close()